# Convolutional Neural Network (CNN) in PyTorch
Sebbene esistano alternative, come la libreria `Keras`, la strategia migliore per scrivere modelli in `PyTorch` è sviluppare sotto classi di una classe che rappresenta __moduli generici__. 
I moduli possono contenere altri moduli, permettendo di annidare elementi funzionali. Le operazioni più comuni, come layer lineari, convoluzionali o self attention etc, sono tutti moduli in PyTorch.

In generale, il pattern di programmazione di un modulo consiste di due fasi: 
- Nella __prima fase__ si dichiarano i moduli necessari alla creazione del modulo corrente, viene fatto nel costruttore. Si scelgono le dimensioni dei sotto moduli e viene effettuata l'inizializzazione.
- Nella __seconda fase__ si va a sovraccaricare il metodo `foreward()`, che, data un'istanza, ritorna un nuovo tensore risultate dalle operazioni specificate nel modulo.

Le dimensioni dei layer non vengono calcolate automaticamente in PyTorch, a differenza di Keras. Quindi, per evitare errori, è utile disegnare uno __schema del modello__.

<div align="center">
<img src="img_CNN.png" width="350">
</div>

Nella figura mostrata abbiamo 4 __blocchi convoluzionali__ tutti uguali tra loro, i quali vanno in un __flattern__ per poi proseguire in una __classification head__. 
La scelta di appiattire l'ultima feature map perchè, lavorando con galassie, l'oggetto da classificare è quasi sempre al centro. Inoltre, il __collo di bottiglia__ permetterà di ridurre il numero di pesi nella classification head. 
Nel disegno sono presenti valori numerici, rappresentano i valori delle dimensioni dei tensori in ingresso e uscita dei blocchi. La strategia dei blocchi è di raddoppiare le feature maps (primo numero della tripletta), dimezzando la dimensione (gli altri due numeri).

Il __blocco__ è una sequenza di __conv 2D__, __batchnorm__, __ReLu__ e per finire __max pooling 2D__ che dimezza la dimensione. Dato che si tratta di una sequenza posso usare `nn.Sequential()` come contenitore. 

In [1]:
import torch 
from torch import nn

In [6]:
class Block(nn.Module):
    def __init__(self, in_channels, num_filters, kernel_size):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels = in_channels,
                        out_channels = num_filters,
                        kernel_size = kernel_size,
                        padding = "same"),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.Conv2d(in_channels = num_filters, # ingresso uguale a out del layer precedente
                        out_channels = num_filters, 
                        kernel_size = kernel_size,
                        padding = "same"),  
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (2, 2)) # dimezza altezza e larghezza delle immagini
        )
        
    def forward(self, x):
        return self.layers(x)

Il blocco è stato definito, ora posso iniziare a costruire la classe principale.

I primi 4 blocchi, quelli di colore rosso nella figura, sono raggruppati in un `Sequential()`. Da notare che il numero di filtri raddoppia ogni volt, mentre la forma delle feature map si riduce di 1/4.

Il __bottleneck__ è un layer convoluzionale con kernel 1x1, che riduce il numero di feature maps a quello di partenza.
Le feature map vengono appiattite in un vettore lungo 6272 dal __flattern__.

Infine, il __multi layer perceptron__ è un layer lineare con 128 neuroni e attivazione ReLu e forma la __classification head__.


In [3]:
class SimpleCNN(nn.Module):
    def __init__(self, num_filters, mpl_size, num_classes):
        super().__init__()
        self.blocks = nn.Sequential(
            Block(3, num_filters, (5, 5)), 
            Block(num_filters, num_filters*2, (3, 3)),
            Block(num_filters*2, num_filters*4, (3, 3)),
            Block(num_filters*4, num_filters*8, (3, 3))
        )
        
        self.bottle_neck = nn.Conv2d(
            in_channels = num_filters*8,
            out_channels = num_filters,
            kernel_size = (1, 1)
        )
        
        self.flatten = nn.Flatten()
        
        self.mlp = nn.Sequential(
            nn.Linear(in_features = 14*14*num_filters, out_features = mpl_size),
            nn.ReLU()
        )
        
        self.classification_head = nn.Linear(in_features = mpl_size, out_features = num_classes)
        
    def forward(self, x):
        x = self.blocks(x)
        x = self.bottle_neck(x)
        x = self.flatten(x)
        x = self.mlp(x)
        return self.classification_head(x)

In [10]:
import torchsummary

batch_size = 4
input_size = 224
num_classes = 10
num_filters = 32
mpl_size = 128

input_tensor = torch.rand(batch_size, 3, input_size, input_size)

model = SimpleCNN(num_filters, mpl_size, num_classes)

torchsummary.summary(model, input_size=(3, input_size, input_size), device="cpu")


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]           2,432
       BatchNorm2d-2         [-1, 32, 224, 224]              64
              ReLU-3         [-1, 32, 224, 224]               0
            Conv2d-4         [-1, 32, 224, 224]          25,632
       BatchNorm2d-5         [-1, 32, 224, 224]              64
              ReLU-6         [-1, 32, 224, 224]               0
         MaxPool2d-7         [-1, 32, 112, 112]               0
             Block-8         [-1, 32, 112, 112]               0
            Conv2d-9         [-1, 64, 112, 112]          18,496
      BatchNorm2d-10         [-1, 64, 112, 112]             128
             ReLU-11         [-1, 64, 112, 112]               0
           Conv2d-12         [-1, 64, 112, 112]          36,928
      BatchNorm2d-13         [-1, 64, 112, 112]             128
             ReLU-14         [-1, 64, 1

Vediamo che il modello è stato costruito correttamente. Ora posso passare alla fase di training usando il dataset `Galaxy10`.

In [ ]:
num_filters = 32
mpl_size = 256
seed = 1234
test_size = 0.2
batch_size = 128
learning_rate = 0.0002
num_epochs = 100
weight_decay = 0.0005
num_classes = 10

model = SimpleCNN(num_filters, mpl_size, num_classes)

